In [1]:
import numpy as np
import pandas as pd

In [3]:
!pwd

/Users/chenghao/ResearchHub/repos/udao-spark-optimizer/playground/assets/data_stats


For TPCH

In [61]:
# Load data
bm = "tpch"
df = pd.read_csv(f"raw_{bm}_hist.csv")
df.head()

,table,column,dtype,bins,distinct_count
0,customer,c_custkey,bigint,"[1.0, 58629.0, 117724.0, 176935.0, 235656.0, 2...",13828764
1,customer,c_name,string,[],15000000
2,customer,c_address,string,[],15000000
3,customer,c_nationkey,bigint,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...",25
4,customer,c_phone,string,[],15000000


In [62]:
def regroup(original_bounds, n=50):
    """
    Transform an n-sized equi-height histogram into a 50-bin equi-height histogram by interpolating within each bin,
    assuming each bin has an equal number of data points.
    
    Args:
    original_bounds (list): The boundaries of the original histogram bins.
    N (int): The desired number of bins in the new histogram (50 by default).
    
    Returns:
    tuple: Tuple containing the new bin boundaries.
    """
    # Determine the number of original bins
    m = len(original_bounds) - 1
    rate = n / m
    new_bounds = [0] * (n + 1)
    new_bounds[0] = original_bounds[0]
    j = 0 # original bin index
    for i in range(1, n):
        while (j + 1) * rate < i:
            j += 1
        offset = i - j * rate
        # print(f"i: {i}, j: {j}, offset: {offset}")
        new_bounds[i] = original_bounds[j] + offset / rate * (original_bounds[j + 1] - original_bounds[j])
    new_bounds[n] = original_bounds[-1]
    return new_bounds


# Example usage
original_bounds = [1, 3, 4, 5, 6, 8]
new_bounds = regroup(original_bounds, n = 10)
print("New Bounds:", new_bounds)
new_bounds = regroup(original_bounds, n = 2)
print("New Bounds:", new_bounds)
new_bounds = regroup(original_bounds, n = 3)
print("New Bounds:", new_bounds)


New Bounds: [1, 2.0, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 7.0, 8]
New Bounds: [1, 4.5, 8]
New Bounds: [1, 3.666666666666667, 5.333333333333334, 8]


In [63]:
from datetime import datetime

def days_since_epoch(date_str):
    """
    Calculate the number of days from January 1, 1970, to the given date.
    
    Args:
    date_str (str): Date in "yyyy-mm-dd" format.
    
    Returns:
    int: Number of days between 1970-01-01 and the given date.
    """
    epoch_start = datetime(1970, 1, 1)
    given_date = datetime.strptime(date_str, '%Y-%m-%d')
    
    # Calculate the difference in days
    delta = given_date - epoch_start
    return delta.days

def regroup_to_50(dtype, bins_str, distinct_count):
    if distinct_count == 1 or bins_str == "[]":
        return [0.0] * 51
    if dtype == "date":
        bins = list(map(days_since_epoch, bins_str[1:-1].split(", ")))
    else:
        bins = list(map(float, bins_str[1:-1].split(", ")))
    
    if distinct_count < 50:
        ret_bins = bins 
        while len(ret_bins) < 50:
            ret_bins += bins
        ret_bins = sorted(ret_bins[:50])
        return ret_bins
        
    return regroup(bins, n=50)

# Regroup data
df["hists"] = df[["dtype", "bins", "distinct_count"]].apply(lambda x: regroup_to_50(x["dtype"], x["bins"], x["distinct_count"]), axis=1)
df.head()

,table,column,dtype,bins,distinct_count,hists
0,customer,c_custkey,bigint,"[1.0, 58629.0, 117724.0, 176935.0, 235656.0, 2...",13828764,"[1.0, 299788.16, 599683.48, 899274.1599999999,..."
1,customer,c_name,string,[],15000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,customer,c_address,string,[],15000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,customer,c_nationkey,bigint,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...",25,"[0.0, 0.0, 1.0, 1.0, 2.0, 2.0, 3.0, 3.0, 4.0, ..."
4,customer,c_phone,string,[],15000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [64]:
df.to_csv(f"regrouped_{bm}_hist.csv", index=False)

For TPCDS

In [65]:
# Load data
bm = "tpcds"
df = pd.read_csv(f"raw_{bm}_hist.csv")
# Regroup data
df["hists"] = df[["dtype", "bins", "distinct_count"]].apply(lambda x: regroup_to_50(x["dtype"], x["bins"], x["distinct_count"]), axis=1)
df.to_csv(f"regrouped_{bm}_hist.csv", index=False)
